In [1]:
# Notebook 3: Multilingual RAG Implementation - Hindi + Chinese
# Run this AFTER notebook 2x to load questions and run experiments


# Suppress all progress bars and warnings
import os
import warnings

# Suppress sentence transformers progress bars
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import logging
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)

import sys
sys.path.append('..')

import json
import time
from pathlib import Path
import pandas as pd
from datetime import datetime
import openai
from dotenv import load_dotenv
from typing import Dict, List, Tuple, Any, Optional
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity

from src.rag_system import MultilingualRAG
from src.utils import translate_text
from langchain.schema import Document

# SET THE API KEY
OPEN_API_KEY = "YOUR KEY"
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY

if OPEN_API_KEY:
    openai.api_key = OPEN_API_KEY
    print(f"OpenAI API Key set (version: {openai.__version__})")
else:
    print("OpenAI API Key not found")

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

ROOT_DIR = Path('..').resolve()
PROCESSED_DATA_DIR = ROOT_DIR / 'data' / 'processed'
EMBEDDINGS_DIR = ROOT_DIR / 'data' / 'embeddings'
QUESTIONS_DIR = ROOT_DIR / 'data' / 'questions'
RESULTS_DIR = ROOT_DIR / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Root directory: {ROOT_DIR}")
print(f"Results directory: {RESULTS_DIR}")

# Load processed documents
processed_path = PROCESSED_DATA_DIR / 'processed_documents.json'

with open(processed_path, 'r', encoding='utf-8') as f:
    docs_data = json.load(f)

documents = []
for doc_dict in docs_data:
    doc = Document(
        page_content=doc_dict['content'],
        metadata=doc_dict['metadata']
    )
    documents.append(doc)

print(f"Loaded {len(documents)} documents")

# Load questions for both languages
def load_questions(language: str):
    """Load questions from JSON file"""
    questions_file = QUESTIONS_DIR / f'{language}_questions.json'
    if not questions_file.exists():
        print(f"Questions file not found: {questions_file}")
        return {}
    
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions_data = json.load(f)
    
    # Flatten questions from all categories
    all_questions = []
    for category, questions in questions_data.items():
        all_questions.extend(questions)
    
    return all_questions

# Load questions for both languages
hindi_questions = load_questions('hindi')
chinese_questions = load_questions('chinese')

print(f"Loaded {len(hindi_questions)} Hindi questions")
print(f"Loaded {len(chinese_questions)} Chinese questions")

# Show samples
if hindi_questions:
    print(f"\nSample Hindi questions:")
    for i, q in enumerate(hindi_questions[:3], 1):
        print(f"  {i}. {q['hindi']}")
        print(f"     ({q['english']})")

if chinese_questions:
    print(f"\nSample Chinese questions:")
    for i, q in enumerate(chinese_questions[:3], 1):
        print(f"  {i}. {q['chinese']}")
        print(f"     ({q['english']})")

# Initialize RAG system
print("\nInitializing RAG system...")

rag_system = MultilingualRAG(
    embedding_model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    vector_store_type='faiss',
    persist_directory=str(EMBEDDINGS_DIR / 'multilingual'),
    llm_model="gpt-4o"
)

vector_store_path = EMBEDDINGS_DIR / 'multilingual' / 'faiss'
if not vector_store_path.exists():
    print("Creating vector store...")
    rag_system.create_vector_store(documents)
else:
    print("Loading existing vector store...")
    rag_system.load_vector_store()

print("RAG system ready")

print("\n" + "="*60)
print("DEBUGGING: CHECKING VECTOR STORE CONTENT")
print("="*60)

# Test what documents are actually in your vector store
test_queries = [
    "heart disease prevention",
    "diabetes symptoms", 
    "blood pressure medication"
]

for query in test_queries:
    print(f"\nTesting query: '{query}'")
    docs = rag_system.vector_store.similarity_search(query, k=3)
    
    for i, doc in enumerate(docs, 1):
        content_preview = doc.page_content[:200].replace('\n', ' ')
        source = doc.metadata.get('source', 'Unknown')
        
        print(f"  Doc {i}: {content_preview}...")
        print(f"  Source: {source}")
        
        # Check for Spanish content
        spanish_words = ['para', 'del', 'con', 'por', 'las', 'los', 'una', 'puede']
        spanish_count = sum(1 for word in spanish_words if word in content_preview.lower())
        
        if spanish_count >= 2:
            print(f"  ⚠️  SPANISH DETECTED ({spanish_count} indicators)")
        
        print(f"  {'-'*50}")

print("="*60)
print("END VECTOR STORE DEBUG")
print("="*60)

# Enhanced Experiment Runner Class
class MultilingualRAGExperiment:
    """
    Experiment runner comparing two approaches for multilingual healthcare QA:
    1. Multilingual embeddings: Query→Retrieve directly→Generate
    2. Translation-based: Query→English→Retrieve→Generate→Target language
    """
    
    def __init__(self, rag_system, has_openai_key: bool = False):
        self.rag_system = rag_system
        self.has_openai_key = has_openai_key
        self.system_prompt = (
            "You are a helpful healthcare information assistant. "
            "Based on the provided context from FDA and NIH medical sources, "
            "answer the question accurately and concisely. "
            "CRITICAL INSTRUCTIONS:\n"
            "1. IGNORE any Spanish text in the context - only use English information\n"
            "2. For MULTILINGUAL approach: Respond in the SAME LANGUAGE as the question (Hindi/Chinese)\n"
            "3. For TRANSLATION approach: Respond ONLY in English\n"
            "4. If the English content doesn't contain relevant information, say so clearly."
        )
    
    def _retrieve_with_metrics(
        self, 
        query: str,
        use_multilingual: bool,
        k: int = 5
    ) -> Tuple[List, str, float, List[float]]:
        """Retrieve documents with detailed metrics"""
        start = time.time()
        
        self.rag_system.use_multilingual_embeddings = use_multilingual
        
        try:
            docs_with_scores = self.rag_system.vector_store.similarity_search_with_score(
                query, k=k
            )
        except (AttributeError, NotImplementedError):
            docs = self.rag_system.vector_store.similarity_search(query, k=k)
            docs_with_scores = [(doc, 1.0 - i*0.05) for i, doc in enumerate(docs)]
        
        documents = [doc for doc, score in docs_with_scores]
        scores = [float(score) for doc, score in docs_with_scores]
        
        retrieval_time = time.time() - start
        
        return documents, query, retrieval_time, scores
    
    def _generate_with_metrics(
        self,
        question: str,
        documents: List,
        target_language: str = "hi"
    ) -> Tuple[str, float, str, str]:
        """Generate response in target language"""
        start = time.time()
        
        # Updated to handle 4 return values
        response, gen_time, chunks, english_for_eval = self.rag_system.generate_response(
            question, documents, target_language, self.system_prompt, return_english_for_eval=True
        )
        
        total_time = time.time() - start
        return response, total_time, chunks, english_for_eval
    
    def _translate_for_eval(
        self, 
        text: str, 
        src_lang: str = "hi",
        dest_lang: str = "en"
    ) -> Tuple[str, float]:
        """Translate text for evaluation purposes"""
        start = time.time()
        translated = translate_text(text, src=src_lang, dest=dest_lang)
        trans_time = time.time() - start
        
        return translated, trans_time
    
    def run_approach_multilingual(
        self,
        question: str,
        language_code: str,
        k: int = 5
    ) -> Dict[str, Any]:
        """Approach A: Direct multilingual embeddings"""
        result = {}
        
        try:
            # Step 1: Retrieve
            docs, eff_query, ret_time, scores = self._retrieve_with_metrics(
                query=question,
                use_multilingual=True,
                k=k
            )
            
            result['multilingual_retrieval_time'] = ret_time
            result['multilingual_retrieved_chunks'] = [d.page_content[:300] for d in docs]
            result['multilingual_chunk_scores'] = scores
            result['multilingual_chunk_sources'] = [d.metadata.get('source', 'Unknown') for d in docs]
            result['multilingual_effective_query'] = eff_query
            
            # Step 2: Generate (now returns 4 values)
            response, gen_time, chunks, english_for_eval = self._generate_with_metrics(
                question, docs, target_language=language_code
            )
            
            result['multilingual_response'] = response
            result['multilingual_generation_time'] = gen_time
            result['multilingual_chunks'] = chunks
            result['multilingual_response_english_for_eval'] = english_for_eval

            result['multilingual_response_translation_time'] = 0.0  # No translation for direct generation
            
            # No separate translation step needed for multilingual approach!
            result['multilingual_total_time'] = ret_time + gen_time
            
            logging.info(f"  Multilingual: {result['multilingual_total_time']:.2f}s (direct generation)")
            
        except Exception as e:
            logging.error(f"  Multilingual failed: {str(e)}")
            result = self._get_error_result('multilingual', str(e))
        
        return result

    def run_approach_translation(
        self,
        question: str,
        language_code: str,
        k: int = 5
    ) -> Dict[str, Any]:
        """Approach B: Translation-based pipeline"""
        result = {}
        
        try:
            # Step 1: Translate question
            question_en, q_trans_time = self._translate_for_eval(
                question, src_lang=language_code, dest_lang="en"
            )
            
            result['translation_question_english'] = question_en
            result['translation_question_translation_time'] = q_trans_time
            
            # Step 2: Retrieve
            docs, eff_query, ret_time, scores = self._retrieve_with_metrics(
                query=question_en,
                use_multilingual=False,
                k=k
            )
            
            result['translation_retrieval_time'] = ret_time
            result['translation_retrieved_chunks'] = [d.page_content[:300] for d in docs]
            result['translation_chunk_scores'] = scores
            result['translation_chunk_sources'] = [d.metadata.get('source', 'Unknown') for d in docs]
            result['translation_effective_query'] = eff_query
            
            # Step 3: Generate (now returns 4 values)
            response, gen_time, chunks, english_for_eval = self._generate_with_metrics(
                question_en, docs, target_language=language_code
            )
            
            result['translation_response'] = response
            result['translation_generation_time'] = gen_time
            result['translation_chunks'] = chunks
            result['translation_response_english_for_eval'] = english_for_eval
            
            # Translation pipeline includes query translation time + generation (which includes back-translation)
            result['translation_total_time'] = q_trans_time + ret_time + gen_time
            
            logging.info(f"  Translation: {result['translation_total_time']:.2f}s (includes query translation)")
            
        except Exception as e:
            logging.error(f"  Translation failed: {str(e)}")
            result = self._get_error_result('translation', str(e))
        
        return result
    
    def _get_error_result(self, approach: str, error_msg: str) -> Dict[str, Any]:
        """Generate error result dictionary"""
        return {
            f'{approach}_error': error_msg,
            f'{approach}_retrieval_time': 0.0,
            f'{approach}_generation_time': 0.0,
            f'{approach}_total_time': 0.0,
            f'{approach}_response': f"ERROR: {error_msg}",
            f'{approach}_response_english_for_eval': f"ERROR: {error_msg}",
            f'{approach}_retrieved_chunks': [],
            f'{approach}_chunk_scores': [],
            f'{approach}_chunk_sources': [],
            f'{approach}_chunks': "",
        }
    
    def run_single_question(
        self,
        question: str,
        question_id: str,
        language: str,
        language_code: str,
        english_equivalent: Optional[str] = None,
        k: int = 5
    ) -> Dict[str, Any]:
        """Run complete experiment for a single question"""
        result = {
            'question_id': question_id,
            'language': language,
            'language_code': language_code,
            'question': question,
            'question_english_reference': english_equivalent or '',
            'system_prompt': self.system_prompt,
            'num_chunks_retrieved': k,
        }
        
        print(f"  Running Approach A (Multilingual embeddings)...")
        result.update(self.run_approach_multilingual(question, language_code, k))
        
        print(f"  Running Approach B (Translation pipeline)...")
        result.update(self.run_approach_translation(question, language_code, k))
        
        return result
    
    def run_experiments_for_language(
        self,
        questions: List[Dict],
        language: str,
        language_code: str,
        k: int = 5
    ) -> List[Dict]:
        """Run experiments for all questions in one language"""
        results = []
        
        print(f"\nRunning {language} Healthcare RAG Experiments")
        print(f"Total questions: {len(questions)}")
        print("=" * 80)
        
        for idx, q_data in enumerate(questions, 1):
            question = q_data[language_code]
            english_ref = q_data.get('english', '')
            
            print(f"\n[{idx}/{len(questions)}] {question}")
            
            question_id = f"{language}_Q{idx:03d}"
            
            result = self.run_single_question(
                question=question,
                question_id=question_id,
                language=language,
                language_code=language_code,
                english_equivalent=english_ref,
                k=k
            )
            results.append(result)
            
            time.sleep(0.3)
        
        return results

# Initialize experiment runner
experiment_runner = MultilingualRAGExperiment(
    rag_system=rag_system,
    has_openai_key=bool(OPEN_API_KEY)
)

# Run experiments for both languages
all_results = []

# Run Hindi experiments
if hindi_questions:
    print("\n" + "="*80)
    print("RUNNING HINDI EXPERIMENTS")
    print("="*80)
    hindi_results = experiment_runner.run_experiments_for_language(
        questions=hindi_questions,
        language="Hindi",
        language_code="hindi",
        k=5
    )
    all_results.extend(hindi_results)

# Run Chinese experiments
if chinese_questions:
    print("\n" + "="*80)
    print("RUNNING CHINESE EXPERIMENTS")
    print("="*80)
    chinese_results = experiment_runner.run_experiments_for_language(
        questions=chinese_questions,
        language="Chinese", 
        language_code="chinese",
        k=5
    )
    all_results.extend(chinese_results)

# Create DataFrame and save results
df_results = pd.DataFrame(all_results)

# Save main results (NO TIMESTAMP)
output_path = RESULTS_DIR / 'multilingual_rag_results.csv'
df_results.to_csv(output_path, index=False, encoding='utf-8')

print("\n" + "="*80)
print("EXPERIMENTS COMPLETE!")
print("="*80)
print(f"Total experiments run: {len(all_results)}")
print(f"  Hindi: {len([r for r in all_results if r['language'] == 'Hindi'])}")
print(f"  Chinese: {len([r for r in all_results if r['language'] == 'Chinese'])}")
print(f"Results saved to: {output_path}")
print(f"Shape: {df_results.shape}")

# Verify chunks were saved
print("\n" + "="*80)
print("VERIFYING CHUNKS WERE SAVED")
print("="*80)

has_multi_chunks = 'multilingual_chunks' in df_results.columns
has_trans_chunks = 'translation_chunks' in df_results.columns

if has_multi_chunks and has_trans_chunks:
    print("SUCCESS! Chunks were saved!")
    print(f"\nSample multilingual chunk (first 300 chars):")
    print(df_results['multilingual_chunks'].iloc[0][:300] + "...")
    print(f"\nSample translation chunk (first 300 chars):")
    print(df_results['translation_chunks'].iloc[0][:300] + "...")
else:
    print("ERROR: Chunks were NOT saved!")
    if not has_multi_chunks:
        print("  Missing: multilingual_chunks")
    if not has_trans_chunks:
        print("  Missing: translation_chunks")

# Language-specific analysis
print("\n" + "="*80)
print("PRELIMINARY LANGUAGE COMPARISON")
print("="*80)

# Split results by language
hindi_df = df_results[df_results['language'] == 'Hindi']
chinese_df = df_results[df_results['language'] == 'Chinese']

if len(hindi_df) > 0:
    print(f"\nHindi Results ({len(hindi_df)} questions):")
    print(f"  Avg Multilingual Time: {hindi_df['multilingual_total_time'].mean():.2f}s")
    print(f"  Avg Translation Time: {hindi_df['translation_total_time'].mean():.2f}s")
    hindi_speedup = hindi_df['translation_total_time'].mean() / hindi_df['multilingual_total_time'].mean()
    print(f"  Translation Speedup: {hindi_speedup:.2f}x")

if len(chinese_df) > 0:
    print(f"\nChinese Results ({len(chinese_df)} questions):")
    print(f"  Avg Multilingual Time: {chinese_df['multilingual_total_time'].mean():.2f}s")
    print(f"  Avg Translation Time: {chinese_df['translation_total_time'].mean():.2f}s")
    chinese_speedup = chinese_df['translation_total_time'].mean() / chinese_df['multilingual_total_time'].mean()
    print(f"  Translation Speedup: {chinese_speedup:.2f}x")

# Save language-specific results for detailed analysis
if len(hindi_df) > 0:
    hindi_path = RESULTS_DIR / 'hindi_rag_results.csv'
    hindi_df.to_csv(hindi_path, index=False, encoding='utf-8')
    print(f"\nSaved Hindi results to: {hindi_path}")

if len(chinese_df) > 0:
    chinese_path = RESULTS_DIR / 'chinese_rag_results.csv'
    chinese_df.to_csv(chinese_path, index=False, encoding='utf-8')
    print(f"Saved Chinese results to: {chinese_path}")

print("\n" + "="*80)
print("Ready for Notebook 4 and 5!")
print("="*80)
print("\nNext steps:")
print("1. Run Notebook 4 for evaluation and analysis")
print("2. Run Notebook 5 for LLM-as-judge evaluation")
print("3. Compare results across Hindi and Chinese")

OpenAI API Key set (version: 1.35.0)
Root directory: C:\Users\Boris\Desktop\code\multilingual-rag
Results directory: C:\Users\Boris\Desktop\code\multilingual-rag\results
Loaded 16036 documents
Loaded 30 Hindi questions
Loaded 30 Chinese questions

Sample Hindi questions:
  1. इंसुलिन प्रतिरोध क्या है और यह कोशिकाओं में ग्लूकोज के प्रवेश को कैसे प्रभावित करता है?
     (What is insulin resistance and how does it affect glucose entry into cells?)
  2. हृदय रोग के जोखिम पर पारिवारिक इतिहास का क्या प्रभाव पड़ता है?
     (How does family history influence the risk of heart disease?)
  3. धूम्रपान का रक्तचाप और हृदय रोग के जोखिम पर क्या प्रभाव पड़ता है?
     (What impact does smoking have on blood pressure and heart disease risk?)

Sample Chinese questions:
  1. 什么是胰岛素抵抗，它如何影响葡萄糖进入细胞？
     (What is insulin resistance and how does it affect glucose entry into cells?)
  2. 家族史如何影响心脏病的风险？
     (How does family history influence the risk of heart disease?)
  3. 吸烟对血压和心脏病风险有何影响？
     (What impact 

2025-11-01 06:49:32,421 - INFO - Loading faiss with AVX2 support.
2025-11-01 06:49:32,421 - INFO - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2025-11-01 06:49:32,421 - INFO - Loading faiss.
2025-11-01 06:49:32,442 - INFO - Successfully loaded faiss.
2025-11-01 06:49:32,625 - INFO - Loaded faiss vector store


RAG system ready

DEBUGGING: CHECKING VECTOR STORE CONTENT

Testing query: 'heart disease prevention'
  Doc 1: . 1.1 Prevention of Cardiovascular Disease In adult patients without clinically evident coronary heart disease, but with multiple risk factors for coronary heart disease such as age, smoking, hyperten...
  Source: FDA
  --------------------------------------------------
  Doc 2: . Possible treatments may include:Heart-healthy lifestyle changesMedicinesProcedures orsurgeriesCardiac rehabilitationCan heart diseases be prevented?You may be able to lower your risk of certain hear...
  Source: MedlinePlus (NIH)
  --------------------------------------------------
  Doc 3: Summary: Heart disease is a general term that includes many types of heart problems. It's a type of cardiovascular disease, which means heart and blood vessel disease. Heart disease is the leading cau...
  Source: MedlinePlus (NIH)
  --------------------------------------------------

Testing query: 'diabetes symp

2025-11-01 06:49:35,122 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-01 06:49:35,127 - INFO - Translating: hindi -> en (mapped: hi -> en)
2025-11-01 06:49:35,127 - INFO - Text preview: इंसुलिन प्रतिरोध एक ऐसी स्थिति है जिसमें आपकी कोशि...
2025-11-01 06:49:36,630 - INFO - Translation successful: Insulin resistance is a condition in which your ce...
2025-11-01 06:49:36,632 - INFO -   Multilingual: 3.66s (direct generation)
2025-11-01 06:49:36,633 - INFO - Translating: hindi -> en (mapped: hi -> en)
2025-11-01 06:49:36,633 - INFO - Text preview: इंसुलिन प्रतिरोध क्या है और यह कोशिकाओं में ग्लूको...


  Running Approach B (Translation pipeline)...


...


[30/30] 在唑来膦酸注射液的上市后使用中报告了哪些不良反应？
  Running Approach A (Multilingual embeddings)...


2025-11-01 06:55:32,755 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-01 06:55:32,755 - INFO - Translating: chinese -> en (mapped: zh-CN -> en)
2025-11-01 06:55:32,755 - INFO - Text preview: The English content provided does not contain info...
2025-11-01 06:55:33,731 - INFO - Translation successful: The English content provided does not contain info...
2025-11-01 06:55:33,731 - INFO -   Multilingual: 2.07s (direct generation)
2025-11-01 06:55:33,731 - INFO - Translating: chinese -> en (mapped: zh-CN -> en)
2025-11-01 06:55:33,731 - INFO - Text preview: 在唑来膦酸注射液的上市后使用中报告了哪些不良反应？...


  Running Approach B (Translation pipeline)...


2025-11-01 06:55:34,165 - INFO - Translation successful: What adverse reactions have been reported during p...
2025-11-01 06:55:34,189 - INFO - Translating: chinese -> en (mapped: zh-CN -> en)
2025-11-01 06:55:34,189 - INFO - Text preview: What adverse reactions have been reported during p...
2025-11-01 06:55:34,730 - INFO - Translation successful: What adverse reactions have been reported during p...
2025-11-01 06:55:35,726 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-01 06:55:35,742 - INFO - Translating: en -> chinese (mapped: en -> zh-CN)
2025-11-01 06:55:35,743 - INFO - Text preview: The provided context does not contain information ...
2025-11-01 06:55:36,348 - INFO - Translation successful: 所提供的上下文不包含有关唑来膦酸注射液上市后使用期间报告的不良反应的信息。...
2025-11-01 06:55:36,348 - INFO -   Translation: 2.62s (includes query translation)



EXPERIMENTS COMPLETE!
Total experiments run: 60
  Hindi: 30
  Chinese: 30
Results saved to: C:\Users\Boris\Desktop\code\multilingual-rag\results\multilingual_rag_results.csv
Shape: (60, 30)

VERIFYING CHUNKS WERE SAVED
SUCCESS! Chunks were saved!

Sample multilingual chunk (first 300 chars):
. This is a condition in which your cells don't respond normally to insulin. As a result, your body needs more insulin to help the glucose enter your cells. At first, your body makes more insulin to try to get cells to respond

. As a result, your body needs more insulin to help the glucose enter yo...

Sample translation chunk (first 300 chars):
. This is a condition in which your cells don't respond normally to insulin. As a result, your body needs more insulin to help the glucose enter your cells. At first, your body makes more insulin to try to get cells to respond

. As a result, your body needs more insulin to help the glucose enter yo...

PRELIMINARY LANGUAGE COMPARISON

Hindi Results (30 q